## Crear una instancia EC2, ejecutarla, pararla y eliminarla

In [101]:
import boto3
import os
from dotenv import load_dotenv

load_dotenv(override=True)

aws_access_key = os.getenv("aws_access_key_id")
aws_secret_key = os.getenv("aws_secret_access_key")
aws_session_token = os.getenv("aws_session_token")
region = os.getenv("aws_region")

ec2_res = boto3.resource(
    "ec2",
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key,
    aws_session_token=aws_session_token,
    region_name=region
)

ec2_cli = boto3.client(
    "ec2",
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key,
    aws_session_token=aws_session_token,
    region_name=region,
)

def obtener_estado_instancia(id):
    response = ec2_cli.describe_instances(InstanceIds=[id])
    state = response["Reservations"][0]["Instances"][0]["State"]["Name"]
    print(f"Estado actual de {id}: {state}")

#### Crear instancia

In [2]:
instances = ec2_res.create_instances(
    ImageId="ami-08b5b3a93ed654d19",
    InstanceType="t2.micro",
    MinCount=1,
    MaxCount=1,
    TagSpecifications=[
        {
            "ResourceType": "instance",
            "Tags": [{"Key": "Name", "Value": "InstanciaBoto3"}],
        }
    ],
)

instance = instances[0]

print(f"id instancia: {instance.id}")

id instancia: i-08ba4628df822ef87


#### Parar la instancia

In [4]:
response = ec2_cli.stop_instances(InstanceIds=[instance.id])
print(response)

{'StoppingInstances': [{'InstanceId': 'i-08ba4628df822ef87', 'CurrentState': {'Code': 64, 'Name': 'stopping'}, 'PreviousState': {'Code': 16, 'Name': 'running'}}], 'ResponseMetadata': {'RequestId': '545cf9fe-252e-4d4c-8ea0-670483c1ecfc', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '545cf9fe-252e-4d4c-8ea0-670483c1ecfc', 'cache-control': 'no-cache, no-store', 'strict-transport-security': 'max-age=31536000; includeSubDomains', 'content-type': 'text/xml;charset=UTF-8', 'content-length': '411', 'date': 'Mon, 24 Mar 2025 15:34:29 GMT', 'server': 'AmazonEC2'}, 'RetryAttempts': 0}}


In [5]:
obtener_estado_instancia(instance.id)

Estado actual de i-08ba4628df822ef87: stopping


#### Eliminar instancia

In [6]:
response = ec2_cli.terminate_instances(InstanceIds=[instance.id])
state = response["TerminatingInstances"][0]["CurrentState"]["Name"]
print(f"Estado: {state}")

Estado: shutting-down


## Cerar un EBS y asociarlo a un EC2 y añadir una archivo

In [38]:
import boto3

iam = boto3.client("iam")
response = iam.list_instance_profiles()

for profile in response["InstanceProfiles"]:
    print(f"🧾 Profile name: {profile['InstanceProfileName']}")
    for role in profile["Roles"]:
        print(f"    ↪ Rol IAM: {role['RoleName']}")


🧾 Profile name: AmazonSSMRoleForInstancesQuickSetup
🧾 Profile name: deomeo
🧾 Profile name: EC2-SSM-Role
🧾 Profile name: EMR_EC2_DefaultRole
    ↪ Rol IAM: EMR_EC2_DefaultRole
🧾 Profile name: LabInstanceProfile
    ↪ Rol IAM: LabRole


In [102]:
instances = ec2_res.create_instances(
    ImageId="ami-08b5b3a93ed654d19",
    InstanceType="t2.micro",
    IamInstanceProfile={"Name": "LabInstanceProfile"},
    KeyName="mirsa",
    MinCount=1,
    MaxCount=1,
    TagSpecifications=[
        {
            "ResourceType": "instance",
            "Tags": [{"Key": "Name", "Value": "InstanciaBoto3"}],
        }
    ],
)

instance = instances[0]

print(f"id instancia: {instance.id}")

id instancia: i-0250e7b735313be7c


#### Crear y asociar

In [18]:
response = ec2_cli.describe_instances(InstanceIds=[instance.id])
availability_zone = response["Reservations"][0]["Instances"][0]["Placement"]["AvailabilityZone"]

In [19]:
volume_response = ec2_cli.create_volume(
    AvailabilityZone=availability_zone,
    Size=8,
    VolumeType="gp2",
    TagSpecifications=[
        {
            "ResourceType": "volume",
            "Tags": [{"Key": "Name", "Value": "miEBS"}],
        }
    ],
)

print(f"id ebs: {volume_response["VolumeId"]}")

id ebs: vol-09b836c1a90f6b53f


In [20]:
vol_status = ec2_cli.describe_volumes(VolumeIds=[volume_response["VolumeId"]])["Volumes"][0]["State"]
if (vol_status == "available"):
    ec2_cli.attach_volume(
        VolumeId=volume_response["VolumeId"],
        InstanceId=instance.id,
        Device="/dev/sdf"
    )
    print("vinculado")
elif (vol_status == "in-use"):
    print("ya asociado")
else:
    print("ebs no esta disponible aun")

vinculado


#### Anadir archivo (ssh)

In [21]:
response = ec2_cli.describe_instances(InstanceIds=[instance.id])
EC2_PUBLIC_IP = response["Reservations"][0]["Instances"][0]["PublicIpAddress"]
print(EC2_PUBLIC_IP)

SSH_KEY_PATH = "mirsa.pem"
EC2_USER = "ec2-user"

VOLUME_DEVICE = "/dev/xvdf"
MOUNT_POINT = "/mnt/ebs"
LOCAL_FILE_PATH = "skere.txt"  

35.173.133.38


#### Subir el archivo

In [24]:
import paramiko

key = paramiko.RSAKey.from_private_key_file(SSH_KEY_PATH)

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

ssh.connect(EC2_PUBLIC_IP, username=EC2_USER, pkey=key)

sftp = ssh.open_sftp()
sftp.put("skere.txt", f"/home/{EC2_USER}/skere.txt")
sftp.close()

print("Archivo subido con éxito.")
stdin, stdout, stderr = ssh.exec_command(f"ls -l /home/{EC2_USER}/skere.txt")
print(stdout.read().decode())
print(stderr.read().decode())

ssh.close()

Archivo subido con éxito.
-rw-rw-r--. 1 ec2-user ec2-user 60 Mar 24 15:43 /home/ec2-user/skere.txt




## Crear un EFS, montarlo y añadir un archivo

In [30]:
KEY_NAME = "mirsa"

#### Crear efs

In [40]:
efs_client = boto3.client(
    "efs",
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key,
    aws_session_token=aws_session_token,
    region_name=region
)

response = efs_client.create_file_system(
    PerformanceMode="generalPurpose",
    ThroughputMode="bursting",
    Tags=[{"Key": "Name", "Value": "miEFS"}],
)

EFS_ID = response["FileSystemId"]
print(f"id efs: {EFS_ID}")


id efs: fs-0a0deb8f96874a749


#### Montaje

In [41]:
# Obtener la VPC y una subred disponible
vpcs = ec2_cli.describe_vpcs()
vpc_id = vpcs["Vpcs"][0]["VpcId"]

subnets = ec2_cli.describe_subnets(Filters=[{"Name": "vpc-id", "Values": [vpc_id]}])
subnet_id = subnets["Subnets"][0]["SubnetId"]

# Crear punto de acceso para el EFS en la subred
response = efs_client.create_mount_target(
    FileSystemId=EFS_ID,
    SubnetId=subnet_id,
    SecurityGroups=["sg-0e4ebccba24ef9c17"],
)

print(f"Punto de montaje en subred {subnet_id}")

Punto de montaje en subred subnet-0873db01c53d03f44


In [42]:
print(instance.id)

i-01b3e12c7db51a3e1


In [43]:
import time

ssm_client = boto3.client("ssm", region_name=os.getenv("AWS_REGION"))

MOUNT_POINT = "/mnt/efs"

# Comandos para instalar el cliente NFS y montar el EFS
commands = [
    "sudo yum install -y amazon-efs-utils",
    f"sudo mkdir -p {MOUNT_POINT}",
    f"sudo mount -t efs {EFS_ID}:/ {MOUNT_POINT}",
    f"echo '{EFS_ID}:/ {MOUNT_POINT} efs defaults,_netdev 0 0' | sudo tee -a /etc/fstab",
]


In [44]:
waiter = ec2_cli.get_waiter("instance_status_ok")
waiter.wait(InstanceIds=[instance.id])


In [35]:
import boto3

iam = boto3.client("iam")
roles = iam.list_roles()

for role in roles["Roles"]:
    print(role["RoleName"])

AWSServiceRoleForAmazonElasticFileSystem
AWSServiceRoleForAPIGateway
AWSServiceRoleForAWSCloud9
AWSServiceRoleForBackup
AWSServiceRoleForCloudWatchEvents
AWSServiceRoleForElastiCache
AWSServiceRoleForGlobalAccelerator
AWSServiceRoleForOrganizations
AWSServiceRoleForRDS
AWSServiceRoleForSupport
AWSServiceRoleForTrustedAdvisor
c138268a3527251l9013214t1w93229969186-LambdaSLRRole-59rcU2yW9Wxs
EMR_AutoScaling_DefaultRole
EMR_DefaultRole
EMR_EC2_DefaultRole
EMR_Notebooks_DefaultRole
LabRole
myRedshiftRole
RedshiftRole
RoleForLambdaModLabRole
vocareum
vocareum-eventbridge
voclabs


In [36]:
import boto3

iam = boto3.client("iam")

attached = iam.list_attached_role_policies(RoleName="LabRole")
print("🔍 Políticas asociadas a 'LabRole':")
for policy in attached["AttachedPolicies"]:
    print(f"- {policy['PolicyName']}")


🔍 Políticas asociadas a 'LabRole':
- AmazonSSMManagedInstanceCore
- AmazonEKSClusterPolicy
- AmazonEC2ContainerRegistryReadOnly
- AmazonEKSWorkerNodePolicy
- c138268a3527251l9013214t1w932299691865-VocLabPolicy2-FM4j2M3FdvLy
- c138268a3527251l9013214t1w932299691865-VocLabPolicy3-nchaIkrPaKuE
- c138268a3527251l9013214t1w932299691865-VocLabPolicy1-bXRztVcoQlrt


In [45]:
response = ssm_client.send_command(
    InstanceIds=[instance.id],
    DocumentName="AWS-RunShellScript",
    Parameters={"commands": commands},
)
command_id = response["Command"]["CommandId"]
print(f"📤 Comando enviado. Command ID: {command_id}")

📤 Comando enviado. Command ID: 99b63b41-bd9c-4390-a357-bf0ca9f6a02d


In [48]:
create_file_commands = [
    f"echo 'holabotoholabotoholabotoholaboto' | sudo tee {MOUNT_POINT}/holaboto.txt",
    f"ls -l {MOUNT_POINT}"
]

response = ssm_client.send_command(
    InstanceIds=[instance.id],
    DocumentName="AWS-RunShellScript",
    Parameters={"commands": create_file_commands},
)

command_id = response["Command"]["CommandId"]
print(f"Comando enviado para crear archivo. ID: {command_id}")

# Esperar la salida
while True:
    output = ssm_client.get_command_invocation(
        CommandId=command_id,
        InstanceId=instance.id,
    )
    status = output["Status"]
    if status in ["Success", "Failed", "Cancelled", "TimedOut"]:
        print(f"Estado: {status}")
        break

    time.sleep(5)


Comando enviado para crear archivo. ID: e98484cf-2bbd-4557-b9b4-125b7e3fbe69
Estado: Success


In [55]:
verify_command = [
    "ls -l /mnt/efs"
]

response = ssm_client.send_command(
    InstanceIds=[instance.id],
    DocumentName="AWS-RunShellScript",
    Parameters={"commands": verify_command},
)

command_id = response["Command"]["CommandId"]

while True:
    output = ssm_client.get_command_invocation(
        CommandId=command_id,
        InstanceId=instance.id,
    )
    status = output["Status"]
    if status in ["Success", "Failed", "Cancelled", "TimedOut"]:
        print(output["Status"])
        print(output["StandardOutputContent"])
        break

    time.sleep(5)

Success
total 4
-rw-r--r--. 1 root root 33 Mar 24 16:28 holaboto.txt



# Crear un S3 Estándar, crear un cubo y añadir varias carpetas con un objeto que sea un archivo csv con varios datos para trabajar con él a posteriori y obtener le objeto

In [60]:
s3 = boto3.client("s3", region_name=region)

s3.create_bucket(
    Bucket="miesetresdorado",
)

{'ResponseMetadata': {'RequestId': 'CA71JSE6RPNRYV83',
  'HostId': '9Xvwmd75aCp4P/UjXF6C4z0bXPHCorQqTBYeV42rkU9n584znZ24vJT05QG/V2/+b+paLOOb1Eo=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '9Xvwmd75aCp4P/UjXF6C4z0bXPHCorQqTBYeV42rkU9n584znZ24vJT05QG/V2/+b+paLOOb1Eo=',
   'x-amz-request-id': 'CA71JSE6RPNRYV83',
   'date': 'Mon, 24 Mar 2025 17:15:30 GMT',
   'location': '/miesetresdorado',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Location': '/miesetresdorado'}

In [61]:
import pandas as pd

folders = [
    "dataset/ventas/",
    "dataset/clientes/",
    "dataset/productos/"
]

df = pd.DataFrame({
    "Producto": ["A", "B", "C"],
    "Cantidad": [10, 5, 7],
    "Precio": [100.5, 200.0, 150.75]
})
csv_file = "ventas.csv"
df.to_csv(csv_file, index=False)

In [65]:
for folder in folders:
    object_key = folder + csv_file
    s3.upload_file(csv_file, "miesetresdorado", object_key)
    print(f"{object_key}")

dataset/ventas/ventas.csv
dataset/clientes/ventas.csv
dataset/productos/ventas.csv


In [66]:
response = s3.list_objects_v2(Bucket="miesetresdorado", Prefix="dataset/")

In [67]:
import io

object_key = "dataset/ventas/ventas.csv"
obj = s3.get_object(Bucket="miesetresdorado", Key=object_key)
df_leido = pd.read_csv(io.BytesIO(obj["Body"].read()))
print("CSV S3:")
print(df_leido)


CSV S3:
  Producto  Cantidad  Precio
0        A        10  100.50
1        B         5  200.00
2        C         7  150.75


# Crear S3 Estándar - Acceso poco frecuente, crear un cubo y añadir un objeto y obtener le objeto

In [68]:
bucket_name = "miesetresdorado2"

s3 = boto3.client("s3", region_name=region)
s3.create_bucket(Bucket=bucket_name)

{'ResponseMetadata': {'RequestId': 'Z514XXEH9NQBKM38',
  'HostId': '4NHzBYsK78Gnz+58EcVEkL55P0Ctt8qRQ7gBWQezPIQEvUVsGib02q1JuGNThTjeItPR4y8ckxg=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '4NHzBYsK78Gnz+58EcVEkL55P0Ctt8qRQ7gBWQezPIQEvUVsGib02q1JuGNThTjeItPR4y8ckxg=',
   'x-amz-request-id': 'Z514XXEH9NQBKM38',
   'date': 'Mon, 24 Mar 2025 17:21:52 GMT',
   'location': '/miesetresdorado2',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Location': '/miesetresdorado2'}

In [69]:
file_name = "ejemplo.txt"
with open(file_name, "w") as f:
    f.write("ejemploejemploejemploejemploejemplo")

In [70]:
object_key = "docs/ejemplo.txt"

s3.upload_file(
    Filename=file_name,
    Bucket=bucket_name,
    Key=object_key,
    ExtraArgs={"StorageClass": "STANDARD_IA"}
)

In [71]:
response = s3.get_object(Bucket=bucket_name, Key=object_key)
contenido = response["Body"].read().decode("utf-8")
print(contenido)


ejemploejemploejemploejemploejemplo


# Crear S3 Intelligent-Tiering, crear un cubo y añadir un objeto y obtener le objeto

In [72]:
bucket_name = "miesetresdorado3"

s3 = boto3.client("s3", region_name=region)
s3.create_bucket(Bucket=bucket_name)

{'ResponseMetadata': {'RequestId': 'A1F7PEGA9R5WAE8Q',
  'HostId': 'pOV2VC0taBsPcg6hEHIRgw61b+721o0igvBH84hjxEkCL4LdHDaLYJIrkGmTQUn5aeMEKRobNjFGQZ/OAr2F4DVJedAivZs2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'pOV2VC0taBsPcg6hEHIRgw61b+721o0igvBH84hjxEkCL4LdHDaLYJIrkGmTQUn5aeMEKRobNjFGQZ/OAr2F4DVJedAivZs2',
   'x-amz-request-id': 'A1F7PEGA9R5WAE8Q',
   'date': 'Mon, 24 Mar 2025 17:24:38 GMT',
   'location': '/miesetresdorado3',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Location': '/miesetresdorado3'}

In [74]:
file_name = "intelligent.txt"
with open(file_name, "w") as f:
    f.write("intelligentintelligentintelligentintelligent")

In [75]:
object_key = "ejemplos/intelligent.txt"

s3.upload_file(
    Filename=file_name,
    Bucket=bucket_name,
    Key=object_key,
    ExtraArgs={"StorageClass": "INTELLIGENT_TIERING"}
)

In [76]:
response = s3.get_object(Bucket=bucket_name, Key=object_key)
contenido = response["Body"].read().decode("utf-8")
print(contenido)


intelligentintelligentintelligentintelligent


# Crear S3 Glacier, crear un cubo y añadir un objeto y obtener le objeto

In [77]:
bucket_name = "miesetresdorado4"

s3 = boto3.client("s3", region_name=region)
s3.create_bucket(Bucket=bucket_name)

{'ResponseMetadata': {'RequestId': 'BB5CKEV1FSMYVXAA',
  'HostId': '1TvjIXIueHq/9rEAi3/U3MfXBdmG2K4zsctE+zkCHmY+NciyewzPxh2tVgdCd27ppLAA6Rcyuwg=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '1TvjIXIueHq/9rEAi3/U3MfXBdmG2K4zsctE+zkCHmY+NciyewzPxh2tVgdCd27ppLAA6Rcyuwg=',
   'x-amz-request-id': 'BB5CKEV1FSMYVXAA',
   'date': 'Mon, 24 Mar 2025 17:38:39 GMT',
   'location': '/miesetresdorado4',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Location': '/miesetresdorado4'}

In [78]:
file_name = "archivo-glacier.txt"
with open(file_name, "w") as f:
    f.write("glacierglacierglacierglacierglacierglacier")

In [79]:
object_key = "archivos/archivo-glacier.txt"

s3.upload_file(
    Filename=file_name,
    Bucket=bucket_name,
    Key=object_key,
    ExtraArgs={"StorageClass": "GLACIER"}
)

Este script enviaria un correo cuando el archivo se restaure

In [ ]:
import boto3
import time
import smtplib
from email.message import EmailMessage

object_key = "archivos/archivo-glacier.txt"
intervalo = 60 #s

destinatario = "uncorreo"
asunto = "Archivo restaurado"
mensaje = f"El objeto '{object_key}' ya ha sido restaurado y está disponible en el bucket '{bucket_name}'."

s3 = boto3.client("s3")

def objeto_restaurado():
    metadata = s3.head_object(Bucket=bucket_name, Key=object_key)
    restored = metadata.get("Restore", "")
    return "ongoing-request=\"false\"" in restored

def enviar_correo_simulado():
    smtp_server = "smtp.gmail.com"
    smtp_port = 587
    remitente = "tucuenta@gmail.com"
    password = "tu_contraseña"

    msg = EmailMessage()
    msg["Subject"] = asunto
    msg["From"] = remitente
    msg["To"] = destinatario
    msg.set_content(mensaje)

    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        server.login(remitente, password)
        server.send_message(msg)

while True:
    try:
        if objeto_restaurado():
            print("El objeto ha sido restaurado.")
            enviar_correo_simulado()
            break
        else:
            print("Aún no restaurado... esperando otro intento.")
    except Exception as e:
        print(f"Error al consultar el estado: {e}")
    
    time.sleep(intervalo)


# Crear S3 Glacier Deep Archive, crear un cubo y añadir un objeto y obtener le objeto

In [80]:
bucket_name = "miesetresdorado5"

s3 = boto3.client("s3", region_name=region)
s3.create_bucket(Bucket=bucket_name)

{'ResponseMetadata': {'RequestId': 'VADRBKHRQ0T14FZF',
  'HostId': 'n2Y+gtNshDnej7OrZ+gvLSFkgqg0EPls4WOLHTMMZEhMvI5cJvMTHkpY4rU3BqQ5zcoNCuuWXCE5jyPReUxbUs1nQvUDWDH6sp5sOetZq20=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'n2Y+gtNshDnej7OrZ+gvLSFkgqg0EPls4WOLHTMMZEhMvI5cJvMTHkpY4rU3BqQ5zcoNCuuWXCE5jyPReUxbUs1nQvUDWDH6sp5sOetZq20=',
   'x-amz-request-id': 'VADRBKHRQ0T14FZF',
   'date': 'Mon, 24 Mar 2025 17:44:52 GMT',
   'location': '/miesetresdorado5',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Location': '/miesetresdorado5'}

In [81]:
file_name = "deep_archive.txt"
with open(file_name, "w") as f:
    f.write("deep_archivedeep_archivedeep_archivedeep_archivedeep_archive")

In [82]:
object_key = "archivos/deep_archive.txt"

s3.upload_file(
    Filename=file_name,
    Bucket=bucket_name,
    Key=object_key,
    ExtraArgs={"StorageClass": "DEEP_ARCHIVE"}
)

Script que avisa*

In [ ]:
object_key = "archivos/deep_archive.txt"
intervalo = 300 #s

s3 = boto3.client("s3", region_name=region)

try:
    s3.restore_object(
        Bucket=bucket_name,
        Key=object_key,
        RestoreRequest={
            "Days": 1,
            "GlacierJobParameters": {
                "Tier": "Standard"
            }
        }
    )
    print("Restauración solicitada correctamente.")
except s3.exceptions.ClientError as e:
    if "RestoreAlreadyInProgress" in str(e):
        print("ℹRestauración ya en curso.")
    elif "is not archived" in str(e):
        print("El objeto no está archivado. Ya se puede leer.")
    else:
        raise e

print(f"Comprobando cada {intervalo} segundos si el archivo está disponible...")

while True:
    try:
        metadata = s3.head_object(Bucket=bucket_name, Key=object_key)
        restored = metadata.get("Restore", "")

        if "ongoing-request=\"false\"" in restored:
            print("\n✅ El objeto ha sido restaurado correctamente.")
            print("📧 Enviando notificación simulada por correo...\n")

            smtp_server = "smtp.gmail.com"
            smtp_port = 587
            remitente = "tucuenta@gmail.com"
            password = "tu_contraseña"
        
            msg = EmailMessage()
            msg["Subject"] = asunto
            msg["From"] = remitente
            msg["To"] = destinatario
            msg.set_content(mensaje)
        
            with smtplib.SMTP(smtp_server, smtp_port) as server:
                server.starttls()
                server.login(remitente, password)
                server.send_message(msg)
            break
        else:
            print("Aún se está restaurando...")
    except Exception as e:
        print(f"Error al verificar restauración: {e}")

    time.sleep(intervalo)


# Hablitar el control de versiones de S3 mediante comandos y mostrar un ejemplo de un objeto modificado y mostrar dos versiones

In [83]:
bucket_name = "miesetresdorado6"

s3 = boto3.client("s3", region_name=region)
s3.create_bucket(Bucket=bucket_name)

{'ResponseMetadata': {'RequestId': '0TVDHS71MFB6QJNW',
  'HostId': 'Ot0wn9TS2rlwOTaA6X80JS2EnnvwrAFTDw3DK/RIozl4ntQqUpMmaye2vPJJnONWgXpHDtJjlAY=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'Ot0wn9TS2rlwOTaA6X80JS2EnnvwrAFTDw3DK/RIozl4ntQqUpMmaye2vPJJnONWgXpHDtJjlAY=',
   'x-amz-request-id': '0TVDHS71MFB6QJNW',
   'date': 'Mon, 24 Mar 2025 17:50:13 GMT',
   'location': '/miesetresdorado6',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Location': '/miesetresdorado6'}

In [84]:
s3.put_bucket_versioning(
    Bucket=bucket_name,
    VersioningConfiguration={"Status": "Enabled"}
)

{'ResponseMetadata': {'RequestId': 'AN9JWEF38961VGP8',
  'HostId': 'ZJxhQCvXy4kYYHjuRyOA90OLrxg+7YGKOeoi9JVjxszblG+/9caA2qVOhgeuya220TwiqnkRU/Y=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'ZJxhQCvXy4kYYHjuRyOA90OLrxg+7YGKOeoi9JVjxszblG+/9caA2qVOhgeuya220TwiqnkRU/Y=',
   'x-amz-request-id': 'AN9JWEF38961VGP8',
   'date': 'Mon, 24 Mar 2025 17:50:31 GMT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [85]:
file_name = "versionado.txt"
object_key = "documentos/versionado.txt"

with open(file_name, "w") as f:
    f.write("primeraversion")

s3.upload_file(file_name, bucket_name, object_key)

In [86]:
with open(file_name, "w") as f:
    f.write("segundaversion")

s3.upload_file(file_name, bucket_name, object_key)

In [87]:
response = s3.list_object_versions(Bucket=bucket_name, Prefix=object_key)

for v in response.get("Versions", []):
    print(f"- ID: {v['VersionId']} | Última modificación: {v['LastModified']} | Última: {v['IsLatest']}")


- ID: _bqsKnO3I9SBE3iROOcQ0ZBnBf3TW7zN | Última modificación: 2025-03-24 17:51:22+00:00 | Última: True
- ID: TgNbn5X8kj9PJ1HnRPK_nwUebf51hNjV | Última modificación: 2025-03-24 17:51:00+00:00 | Última: False


In [88]:
for v in response.get("Versions", []):
    version_id = v["VersionId"]
    is_latest = v["IsLatest"]

    obj = s3.get_object(Bucket=bucket_name, Key=object_key, VersionId=version_id)
    content = obj["Body"].read().decode("utf-8")

    print(f"\nVersión ID: {version_id} {'(última)' if is_latest else ''}")
    print(content)



Versión ID: _bqsKnO3I9SBE3iROOcQ0ZBnBf3TW7zN (última)
segundaversion

Versión ID: TgNbn5X8kj9PJ1HnRPK_nwUebf51hNjV 
primeraversion


# Realizar 3 consultas diferentes sobre el objeto .csv del S3 usando AWS Athena

In [98]:
athena = boto3.client("athena", region_name=region)

s3_csv_path = "s3://miesetresdorado/dataset/ventas/ventas.csv"
s3_output = "s3://mi-bucket-datos/athena-results/"
database = "default"


In [103]:
create_table_query = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS ventas (
    producto STRING,
    cantidad INT,
    precio DOUBLE
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES ('serialization.format' = ',', 'field.delim' = ',')
LOCATION '{s3_csv_path}'
TBLPROPERTIES ('has_encrypted_data'='false');
"""

athena.start_query_execution(
    QueryString=create_table_query,
    QueryExecutionContext={"Database": database},
    ResultConfiguration={"OutputLocation": s3_output},
)

ClientError: An error occurred (AccessDeniedException) when calling the StartQueryExecution operation: You are not authorized to perform: athena:StartQueryExecution on the resource. After your AWS administrator or you have updated your permissions, please try again.

error de permisos (?)